# Access Derive

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
cd '/content/drive/MyDrive/Colab Notebooks'


/content/drive/MyDrive/Colab Notebooks


# Read Data

In [31]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as pyplot
import os

In [85]:
URM_train=pd.read_csv('input/data_train.csv')
R_test=pd.read_csv('input/data_target_users_test.csv')


In [86]:
URM=pd.read_csv('input/data_train.csv')
URM_tuples = [tuple(x) for x in URM.to_numpy()]


In [87]:

URM_tuples = [tuple(x) for x in URM_train.to_numpy()]


In [88]:
userList, itemList, ratingList = zip(*URM_tuples)

userList = list(userList)
userList=np.array(userList,dtype=np.int64)
itemList = list(itemList)
itemList=np.array(itemList,dtype=np.int64)

ratingList = list(ratingList)                        #not needed
ratingList=np.array(ratingList,dtype=np.int64)       #not needed





In [89]:
len(ratingList)-np.count_nonzero(ratingList)

0

In [90]:
itemList

array([10080, 19467,  2665, ..., 21384,  8699, 19178])

In [91]:
userList_unique = list(set(userList))
itemList_unique = list(set(itemList))

numUsers = len(userList_unique)
numItems = len(itemList_unique)

numberInteractions= len(URM_tuples)
print ("Number of items\t {}, Number of users\t {}".format(numItems, numUsers))
print("Number of Intraction \t {}" .format(numberInteractions))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemList_unique), max(userList_unique)))
print ("Average interactions per user {:.2f}".format(numberInteractions/numUsers))
print ("Average interactions per item {:.2f}\n".format(numberInteractions/numItems))

print ("Sparsity {:.2f} %".format((1-float(numberInteractions)/(numItems*numUsers))*100))

Number of items	 24896, Number of users	 7947
Number of Intraction 	 113268
Max ID items	 25974, Max Id users	 7946

Average interactions per user 14.25
Average interactions per item 4.55

Sparsity 99.94 %


To store the data we use a sparse matrix. We build it as a COO matrix and then change its format
The COO constructor expects (data, (row, column))

In [92]:

import scipy.sparse as sps

URM_all = sps.coo_matrix((ratingList, (userList, itemList)))

URM_all

<7947x25975 sparse matrix of type '<class 'numpy.int64'>'
	with 113268 stored elements in COOrdinate format>

In [93]:
URM_all.tocsr()

<7947x25975 sparse matrix of type '<class 'numpy.longlong'>'
	with 113268 stored elements in Compressed Sparse Row format>

** Item popularity
**

In [94]:
import numpy as np

itemPopularity = (URM_all>0).sum(axis=0)
itemPopularity

matrix([[2, 3, 5, ..., 1, 1, 2]], dtype=int64)

In [95]:
itemPopularity = np.array(itemPopularity).squeeze()
itemPopularity

array([2, 3, 5, ..., 1, 1, 2], dtype=int64)

In [96]:
itemPopularity = np.sort(itemPopularity)
itemPopularity

array([  0,   0,   0, ..., 219, 234, 393], dtype=int64)

# Split data for test and train

In [132]:
train_test_split = 0.999

numInteractions = URM_all.nnz


train_mask = np.random.choice([True,False], numInteractions, p=[train_test_split, 1-train_test_split])
train_mask

array([ True,  True,  True, ...,  True,  True,  True])

In [133]:
userList = np.array(userList)
itemList = np.array(itemList)
ratingList = np.array(ratingList)


URM_train = sps.coo_matrix((ratingList[train_mask], (userList[train_mask], itemList[train_mask])))   #changed variable specification
URM_train = URM_train.tocsr()
URM_train

<7947x25975 sparse matrix of type '<class 'numpy.longlong'>'
	with 113138 stored elements in Compressed Sparse Row format>

In [135]:
test_mask = np.logical_not(train_mask)

URM_test = sps.coo_matrix((ratingList[test_mask], (userList[test_mask], itemList[test_mask])))
URM_test = URM_test.tocsr()
URM_test

<7900x25819 sparse matrix of type '<class 'numpy.longlong'>'
	with 130 stored elements in Compressed Sparse Row format>

# Evaluation Metrics

In [136]:
user_id = userList_unique[1]
user_id

1

In [137]:
 from Base.Evaluation import Evaluator

In [138]:
#from Data_manager.Movielens.Movielens10MReader import Movielens10MReader
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [139]:
#dataReader = Movielens10MReader()
#dataset = dataReader.load_data()

#URM_all = dataset.get_URM_all()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.999)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.999)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

# Run on Train and Test

In [140]:
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
#from KNN import *
from KNN import ItemKNNCFRecommender as ICF

In [141]:
best_parameters = {'topK': 10, 'shrink': 10, 'similarity': 'cosine', 'normalize': True}

itemKNNCF = ICF.ItemKNNCFRecommender(URM_train)
itemKNNCF.fit(**best_parameters)

ItemKNNCFRecommender: URM Detected 1089 (4.19 %) cold items.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 25975 ( 100 % ), 2135.21 column/sec, elapsed time 0.20 min


In [142]:
from GraphBased.P3alphaRecommender import P3alphaRecommender

P3alpha = P3alphaRecommender(URM_train)
P3alpha.fit()

P3alphaRecommender: URM Detected 1089 (4.19 %) cold items.


In [143]:
itemKNNCF.W_sparse

<25975x25975 sparse matrix of type '<class 'numpy.float32'>'
	with 240538 stored elements in Compressed Sparse Row format>

In [144]:

P3alpha.W_sparse

<25975x25975 sparse matrix of type '<class 'numpy.float32'>'
	with 1285285 stored elements in Compressed Sparse Row format>

In [145]:
from KNN import ItemKNNSimilarityHybridRecommender as Hyb

In [146]:
hybridrecommender = Hyb.ItemKNNSimilarityHybridRecommender(URM_train, itemKNNCF.W_sparse, P3alpha.W_sparse)

ItemKNNSimilarityHybridRecommender: URM Detected 1089 (4.19 %) cold items.


In [147]:
for i in range(60, 150, 10):
    hybridrecommender.fit(topK = i,  alpha = 0.36) #k = 90
    temp =  evaluator_validation.evaluateRecommender(hybridrecommender)
    print("i = " + str(i) + "   map = " + str(temp[0][10]['MAP']))

EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1526
i = 60   map = 0.039390756302521014
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1519
i = 70   map = 0.039390756302521014
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1458
i = 80   map = 0.039390756302521014
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1501
i = 90   map = 0.034488795518207285
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1459
i = 100   map = 0.04024665421724245
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1428
i = 110   map = 0.040301120448179274
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1494
i = 120   map = 0.040301120448179274
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per second: 1458
i = 130   map = 0.040301120448179274
EvaluatorHoldout: Processed 102 ( 100.00% ) in 0.07 sec. Users per se

# Output

In [148]:
hybridrecommender.fit(topK = 105,  alpha = 0.36) #k = 105

In [ ]:

for user_id in range():
    print(topPopRecommender.recommend(user_id, at=10))

In [177]:
#R_test=pd.read_csv('input/data_target_users_test.csv')
n = len(R_test)

azina = []
for i in range(n):
  usr_id=R_test['user_id'][i]
  str_temp = str(hybridrecommender.recommend(usr_id,10))[1:-1]
  str_temp = str_temp.replace(',','')
  R_test['item_list'][i]=str_temp


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [181]:
R_test=R_test.set_index('user_id')

In [182]:
R_test

,item_list
user_id,
0,5085 8486 1789 19062 4927 1447 25878 6961 1511...
1,23600 12409 19089 19709 8894 8431 23244 3165 1...
2,15427 14031 21626 9547 2824 4842 11548 13251 9...
3,12473 6649 22487 3942 16834 7749 9528 3 2074 1...
4,1611 9243 19843 9578 9295 12474 19704 4294 185...
...,...
7942,24398 25755 24764 4068 25338 11445 20130 2458 ...
7943,14137 20368 19874 25760 4502 7913 24348 18419 ...
7944,12116 6220 24093 24636 857 3338 2562 17820 135...


In [183]:
R_test.to_csv('result_itemLinearHybrid.csv')